In [1]:
from google.cloud import speech
from google.cloud import texttospeech

import pvporcupine
import pyaudio
import struct
import subprocess

import numpy as np



from dotenv import load_dotenv
import os

import tkinter as tk
import pyaudio
import wave
from datetime import datetime
import sounddevice as sd
import speech_recognition as sr

In [2]:
# Creds

load_dotenv()
pico_key = os.getenv("picokey")

os.environ["BING_U_COOKIE"] = os.getenv("BING_COOKIE")

In [3]:
# Functions

porcupine = pvporcupine.create(
  access_key=pico_key,
  keyword_paths = ['./VoiceRecognition/Hey-Jade.ppn']
)

# Porcupine Wake Word
def get_next_audio_frame():
    pa = pyaudio.PyAudio()

    audio_stream = pa.open(
                    rate=porcupine.sample_rate,
                    channels=1,
                    format=pyaudio.paInt16,
                    input=True,
                    frames_per_buffer=porcupine.frame_length)
    return audio_stream

# Voice Recognition
def VoiceToText(speech_file: str) -> speech.RecognizeResponse:

    """Transcribe the given audio file."""
    client = speech.SpeechClient()

    with open(speech_file, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code="en-US",
    )

    response = client.recognize(config=config, audio=audio)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print(f"Transcript: {result.alternatives[0].transcript}")

    return result.alternatives[0].transcript

# Text To Voice
def texttoVoice(AiText):
    # Instantiates a client
    client = texttospeech.TextToSpeechClient()

    # Set the text input to be synthesized
    synthesis_input = texttospeech.SynthesisInput(text=AiText)

    # Build the voice request, select the language code ("en-US") and the ssml
    # voice gender ("neutral")
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )

    # Select the type of audio file you want returned
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    # Perform the text-to-speech request on the text input with the selected
    # voice parameters and audio file type
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    # The response's audio_content is binary.
    with open("output.mp3", "wb") as out:
        # Write the response to the output file.
        out.write(response.audio_content)
        print('Audio content written to file "output.mp3"') 


# Call Bing
def callBing(prompt):

    node_script = 'gptbing.js'
    arguments = [prompt]

    command = ['node',node_script] + arguments

    try:
        result = subprocess.run(command, check=True,stdout=subprocess.PIPE)
        output = result.stdout.decode('utf-8')
        return output
    except subprocess.CalledProcessError as e:
        print('Error:', e)
        print('Output:', e.output)

def recordAudionSave ():
    # Initialize the recognizer
    recognizer = sr.Recognizer()

    # Initialize the microphone
    microphone = sr.Microphone()

    # Create a function to record audio and save it to a WAV file
    def record_audio_and_save(filename):
        with microphone as source:
            print("Listening... (press Ctrl+C to stop)")
            try:
                audio = recognizer.listen(source, timeout=20, phrase_time_limit=20)
            except sr.WaitTimeoutError:
                print("No speech detected. Recording stopped.")
                return

        # Save the audio to a WAV file
        with open(filename, "wb") as file:
            file.write(audio.get_wav_data())

        print(f"Audio saved as {filename}")

    if __name__ == "__main__":
        try:
            # Specify the filename to save the recorded audio
            audio_filename = "recorded_audio.wav"
            
            # Record audio once and then exit
            record_audio_and_save(audio_filename)
        except KeyboardInterrupt:
            print("Recording stopped.")


In [4]:
while True:
  audio_stream = get_next_audio_frame()
  pcm = audio_stream.read(porcupine.frame_length)
  pcm = struct.unpack_from("h" * porcupine.frame_length, pcm)

  keyword_index = porcupine.process(pcm)
  
  if keyword_index == 0:
    recordAudionSave()
    print("Listening...")
    prompt = VoiceToText("recorded_audio.wav")
    output = callBing(prompt)
    print(output)
    texttoVoice(output)
    break
  elif keyword_index == 1:
    print("Nothing...")
      

Listening... (press Ctrl+C to stop)
Audio saved as recorded_audio.wav
Listening...


UnboundLocalError: local variable 'result' referenced before assignment

Listening... (press Ctrl+C to stop)
Audio saved as recorded_audio.wav
